# 读取数据

In [20]:
import pandas as pd
import pyodbc
import numpy as np
import math
from sqlalchemy import create_engine
engine = create_engine('mssql+pyodbc://sa:suzhou@.\SQLEXPRESS/POIs?driver=SQL server')

In [3]:
with engine.connect() as conn,conn.begin():
    data = pd.read_sql_table('POI_X',conn)

C:\Users\workload\Anaconda3\lib\site-packages\sqlalchemy\dialects\mssql\base.py:2148: SAWarning: Did not recognize type 'geometry' of column '形状'
  (type, name))


In [4]:
data.head()

,OBJECTID,形状,Join_Count,TARGET_FID,Field1,Unnamed__0,index_,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84,Shape_Length,Shape_Area,in_check
0,1,b'\x00\x00\x00\x00\x01\x0c\x00\x00\x00\x00\xaa...,0,1,0,0,0,江心洲农家乐,31.611946,118.459650,商业,酒店,31.607511,118.447853,NaN,NaN,NaN
1,2,b'\x00\x00\x00\x00\x01\x0c\x00\x00\x00\x00\xee...,4,2,1,1,1,TIAN,31.676164,118.495139,商业,购物,31.672398,118.483323,0.008057,0.000004,11742.0
2,3,b'\x00\x00\x00\x00\x01\x0c\x00\x00\x00\x00u\xa...,4,3,2,2,2,欧派铁艺,31.978026,118.737678,商业,购物,31.974367,118.725875,0.007935,0.000004,70904.0
3,4,b'\x00\x00\x00\x00\x01\x0c\x00\x00\x00\x00\x02...,4,4,3,3,3,飞翔超市,31.615439,118.496189,商业,购物,31.611680,118.484360,0.008057,0.000004,12.0
4,5,b'\x00\x00\x00\x00\x01\x0c\x00\x00\x00\x00\xc4...,2,5,4,4,4,采当车行,31.613138,118.492454,商业,购物,31.609326,118.480621,0.008057,0.000004,10.0


In [5]:
len(data)

307153

# 筛选数据，只需要我们的in_check

In [6]:
data = data[['in_check','sort1']]

In [7]:
dtgb = data.groupby(by='in_check')

In [8]:
len(dtgb)

24724

# 按13个分类生成列

In [14]:
dm = dtgb['sort1'].value_counts().unstack().fillna(0)

# 计算熵值

In [16]:
dx = dm.copy()

In [17]:
dx.columns

Index(['交通服务设施', '停车场', '公共服务', '医疗', '商业', '商务办公', '娱乐', '工业', '教育', '文化设施',
       '旅游景点', '行政机构', '餐饮'],
      dtype='object', name='sort1')

In [18]:
dx['all'] = dx['交通服务设施'] + dx['停车场'] + dx['公共服务'] + dx['医疗'] + dx['商业'] + \
dx['商务办公'] + dx['娱乐'] + dx['工业'] + dx['教育'] + dx['文化设施'] + dx['旅游景点'] + dx['行政机构'] + dx['餐饮']

In [19]:
dx.head()

sort1,交通服务设施,停车场,公共服务,医疗,商业,商务办公,娱乐,工业,教育,文化设施,旅游景点,行政机构,餐饮,all
in_check,,,,,,,,,,,,,,
10.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
12.0,2.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,12.0
13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
16.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0


In [21]:
dx.reset_index(inplace=True)

In [22]:
dx.head()

sort1,in_check,交通服务设施,停车场,公共服务,医疗,商业,商务办公,娱乐,工业,教育,文化设施,旅游景点,行政机构,餐饮,all
0,10.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,12.0,2.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,12.0
2,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
3,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,16.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
dx[['交通服务设施','all']]

In [23]:
def get_s(x):
    if x[1] == 0:
        return 0
    elif x[0] == 0:
        return 0
    else:
        return (-x[0]/x[1]) * (math.log(x[0]/x[1]))

In [24]:
dx['交通服务设施S'] = dx[['交通服务设施','all']].apply(get_s,axis=1)

In [25]:
dx.head()

sort1,in_check,交通服务设施,停车场,公共服务,医疗,商业,商务办公,娱乐,工业,教育,文化设施,旅游景点,行政机构,餐饮,all,交通服务设施S
0,10.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.346574
1,12.0,2.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,12.0,0.298627
2,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000000
3,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000
4,16.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,0.366516


In [26]:
pois = ['交通服务设施', '停车场', '公共服务', '医疗', '商业', '商务办公', '娱乐', '工业', '教育', '文化设施',
       '旅游景点', '行政机构', '餐饮']

In [27]:
for i in pois:
    dx[i+'S'] = dx[[i,'all']].apply(get_s,axis=1)

In [28]:
dx.head()

sort1,in_check,交通服务设施,停车场,公共服务,医疗,商业,商务办公,娱乐,工业,教育,...,医疗S,商业S,商务办公S,娱乐S,工业S,教育S,文化设施S,旅游景点S,行政机构S,餐饮S
0,10.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.346574,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,12.0,2.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.207076,0.0,0.0,0.346574,0.0,0.0,0.0,0.0,0.298627
2,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.000000,0.0,0.0,-0.000000,0.0,0.0,0.0,0.0,0.000000
3,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.000000,0.0,0.0,-0.000000,0.0,0.0,0.0,0.0,0.000000
4,16.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,...,0.0,0.321888,0.0,0.0,0.366516,0.0,0.0,0.0,0.0,0.000000


In [29]:
dx['allS'] = 0
for i in pois:
    dx['allS'] += dx[i+'S']
    

In [30]:
dx.head()

sort1,in_check,交通服务设施,停车场,公共服务,医疗,商业,商务办公,娱乐,工业,教育,...,商业S,商务办公S,娱乐S,工业S,教育S,文化设施S,旅游景点S,行政机构S,餐饮S,allS
0,10.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.346574,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.039721
1,12.0,2.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,...,0.207076,0.0,0.0,0.346574,0.0,0.0,0.0,0.0,0.298627,1.517106
2,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.000000,0.0,0.0,-0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
3,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.0,0.0,-0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
4,16.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,...,0.321888,0.0,0.0,0.366516,0.0,0.0,0.0,0.0,0.000000,1.054920


In [31]:
dx = dx[['in_check','allS']]

In [32]:
dx.head()

sort1,in_check,allS
0,10.0,1.039721
1,12.0,1.517106
2,13.0,0.000000
3,15.0,0.000000
4,16.0,1.054920


In [33]:
dx.to_csv(r'D:\Vgis_data\Multi-Source-Data\show\poi_ss.csv')